In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
display_table('Queue')

+-----------+-------------+--------+------+
| person_id | person_name | weight | turn |
+-----------+-------------+--------+------+
|     5     |    Alice    |  250   |  1   |
|     4     |     Bob     |  175   |  5   |
|     3     |     Alex    |  350   |  2   |
|     6     |  John Cena  |  400   |  3   |
|     1     |   Winston   |  500   |  6   |
|     2     |    Marie    |  200   |  4   |
+-----------+-------------+--------+------+


### There is a queue of people waiting to board a bus. However, the bus has a weight limit of 1000 kilograms, so there may be some people who cannot board.

Write an SQL query to find the person_name of the last person that can fit on the bus without exceeding the weight limit. The test cases are generated such that the first person does not exceed the weight limit.

```
+-------------+
| person_name |
+-------------+
| John Cena   |
+-------------+
Explanation: The folowing table is ordered by the turn for simplicity.
+------+----+-----------+--------+--------------+
| Turn | ID | Name      | Weight | Total Weight |
+------+----+-----------+--------+--------------+
| 1    | 5  | Alice     | 250    | 250          |
| 2    | 3  | Alex      | 350    | 600          |
| 3    | 6  | John Cena | 400    | 1000         | (last person to board)
| 4    | 2  | Marie     | 200    | 1200         | (cannot board)
| 5    | 4  | Bob       | 175    | ___          |
| 6    | 1  | Winston   | 500    | ___          |
+------+----+-----------+--------+--------------+
```


In [27]:
%%sql 

SET @total_weight := 0;

SELECT 
    weight,
    @total_weight,
    (@total_weight := @total_weight + weight) AS weight_sum
FROM Queue 
ORDER BY turn

weight,@total_weight,weight_sum
250,0,250
350,250,600
400,600,1000
200,1000,1200
175,1200,1375
500,1375,1875


In [34]:
%%sql 

SET @total_weight := 0;

SELECT 
    person_name,
    weight_sum
FROM (
    SELECT
        person_id,
        person_name,
        (@total_weight := @total_weight + weight) AS weight_sum
    FROM Queue
    ORDER BY turn
)AS inner_table
WHERE inner_table.weight_sum <= 1000

person_name,weight_sum
Alice,250
Alex,600
John Cena,1000


In [39]:
%%sql 

SET @total_weight := 0;

SELECT 
    person_name
FROM (
    SELECT
        person_id,
        person_name,
        (@total_weight := @total_weight + weight) AS weight_sum,
        turn
    FROM Queue
    ORDER BY turn
) AS inner_table
WHERE weight_sum <= 1000
ORDER BY turn DESC;

person_name
John Cena
Alex
Alice


In [40]:
%%sql 

SET @total_weight := 0;

SELECT 
    person_name
FROM (
    SELECT
        person_id,
        person_name,
        (@total_weight := @total_weight + weight) AS weight_sum,
        turn
    FROM Queue
    ORDER BY turn
) AS inner_table
WHERE weight_sum <= 1000
ORDER BY turn DESC
LIMIT 1;

person_name
John Cena


# Using Pandas

In [11]:
queue_query = %sql SELECT * FROM Queue # type: ignore
queue_df = queue_query.DataFrame()

display(queue_df)

,person_id,person_name,weight,turn
0,5,Alice,250,1
1,4,Bob,175,5
2,3,Alex,350,2
3,6,John Cena,400,3
4,1,Winston,500,6
5,2,Marie,200,4


In [14]:
queue_df = queue_df.sort_values(by='turn')
queue_df

,person_id,person_name,weight,turn
0,5,Alice,250,1
2,3,Alex,350,2
3,6,John Cena,400,3
5,2,Marie,200,4
1,4,Bob,175,5
4,1,Winston,500,6


In [22]:
queue_df['weight'].cumsum()

0     250
2     600
3    1000
5    1200
1    1375
4    1875
Name: weight, dtype: int64

In [43]:
queue_df['weight_sum'] = queue_df['weight'].cumsum()
queue_df

,person_id,person_name,weight,turn,previous_weight,weight_sum
0,5,Alice,250,1,NaN,250
2,3,Alex,350,2,250.0,600
3,6,John Cena,400,3,350.0,1000
5,2,Marie,200,4,400.0,1200
1,4,Bob,175,5,200.0,1375
4,1,Winston,500,6,175.0,1875


In [44]:
queue_df.query("weight_sum <= 1000")

,person_id,person_name,weight,turn,previous_weight,weight_sum
0,5,Alice,250,1,NaN,250
2,3,Alex,350,2,250.0,600
3,6,John Cena,400,3,350.0,1000


In [55]:
help(queue_df.idxmax)

Help on method idxmax in module pandas.core.frame:

idxmax(axis: 'Axis' = 0, skipna: 'bool' = True, numeric_only: 'bool' = False) -> 'Series' method of pandas.core.frame.DataFrame instance
    Return index of first occurrence of maximum over requested axis.
    
    NA/null values are excluded.
    
    Parameters
    ----------
    axis : {0 or 'index', 1 or 'columns'}, default 0
        The axis to use. 0 or 'index' for row-wise, 1 or 'columns' for column-wise.
    skipna : bool, default True
        Exclude NA/null values. If an entire row/column is NA, the result
        will be NA.
    numeric_only : bool, default False
        Include only `float`, `int` or `boolean` data.
    
        .. versionadded:: 1.5.0
    
    Returns
    -------
    Series
        Indexes of maxima along the specified axis.
    
    Raises
    ------
    ValueError
        * If the row/column is empty
    
    See Also
    --------
    Series.idxmax : Return index of the maximum element.
    
    Notes
 

In [65]:
max_index = queue_df.query("weight_sum <= 1000").index.max()
max_index

3

In [66]:
queue_df.query("index == @max_index")

,person_id,person_name,weight,turn,previous_weight,weight_sum
3,6,John Cena,400,3,350.0,1000


In [67]:
queue_df.query("index == @max_index")[['person_name']]

,person_name
3,John Cena
